In [5]:
import pandas as pd
import numpy as np
import json
import os

import git
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

In [2]:
folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQD\Optimize\VaryBeta")
comp='beta'
potentials = ['QHO','AHO','DW']
cutoffs = [2,4,8,16]
converged_only = True
best_values = []
alld = []

for potential in potentials: 

    folders = [f for f in os.listdir(os.path.join(folder_path,potential)) if os.path.isdir(os.path.join(folder_path, potential, f))]

    for cutoff in cutoffs:
        all_data = []

        for folder in folders:   

            #print(f"Runing for {potential} potential, {cutoff} cutoff and {folder} value")
            
            #if folder == '100000': print('######################')

            median = []
            mean = []
            min = []
            e0 = []
            e1 = []
            e2 = []
            ratios = []

            data_path = os.path.join(folder_path, potential, str(folder), "{}_{}.json".format(potential, cutoff))
            
            with open(data_path, 'r') as file:
                data = json.load(file)

           
            num_VQD = data['num_VQD']

            converged = [all(x) for x in data['success']]
            converged_indices = [i for i, success in enumerate(converged) if success]
            converged_runs = sum(converged)

            if converged_only:
                results = [data['results'][i] for i in converged_indices]
            else:
                results = data['results']

            success_i = [i for i, success in enumerate(data['success']) if all(success)]
            successful_results = [data['results'][i] for i in success_i]
            energies =np.sort(successful_results)

            
            run_times = data['run_times']
            time_series = np.array([pd.Timedelta(t).total_seconds() for t in run_times])
            mean_vqe_time = np.mean(time_series)
            time_taken = np.mean(mean_vqe_time)

            for e in np.sort(energies):
                e0.append(e[0])
                e1.append(e[1])
                e2.append(e[2])

            mean_ratio = abs((np.mean(e2) - np.mean(e1)) / (np.mean(e2) - np.mean(e0)))
            median_ratio = abs((np.median(e2) - np.median(e1)) / (np.median(e2) - np.median(e0)))

            e0_exact = data['exact_eigenvalues'][0]
            e1_exact = data['exact_eigenvalues'][1]
            e2_exact = data['exact_eigenvalues'][2]
            exact_ratio = abs((e2_exact - e1_exact) / (e2_exact - e0_exact))

            row = {
            comp: folder,
            "potential": potential,
            "cutoff": cutoff,
            'Converged Runs': f"{converged_runs}/100",
            'Mean VQD time (s)': f"{time_taken:.2f}",
            "e0": np.median(e0),
            "e1": np.median(e1),
            "e2": np.median(e2),
            "median_ratio": median_ratio,
            "mean_ratio": mean_ratio,
            #"min_ratio": np.min(ratios),
            "e0_exact": e0_exact,
            "e1_exact": e1_exact,
            "e2_exact": e2_exact,
            "exact_ratio": exact_ratio
            }

            if folder == '100000': print(row)

            all_data.append(row)
            alld.append(row)

        
        df = pd.DataFrame(all_data.copy())
        df['diff'] = df[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
        df.sort_values('diff').to_excel(os.path.join(folder_path, potential, f"{potential}{cutoff}.xlsx"), index=False)
        #df.to_csv(os.path.join(folder_path, potential, f"{potential}{cutoff}.csv"), float_format="%.10f")

        best = df.sort_values('diff').iloc[0][comp]

        d = {"potential": potential,
            "cutoff": cutoff,
            "best value": best}

        best_values.append(d)

In [3]:
df

,beta,potential,cutoff,Converged Runs,Mean VQD time (s),e0,e1,e2,median_ratio,mean_ratio,e0_exact,e1_exact,e2_exact,exact_ratio,diff
0,0.5,DW,16,99/100,322.26,0.893699,0.914790,1.414790,0.959526,0.801797,0.891599,0.891641,2.734122,0.999977,0.040452
1,1.0,DW,16,100/100,278.48,0.893699,0.914893,1.914707,0.979243,0.934378,0.891599,0.891641,2.734122,0.999977,0.020735
2,10.0,DW,16,100/100,183.61,0.893699,0.914790,3.130247,0.990570,0.963139,0.891599,0.891641,2.734122,0.999977,0.009407
3,2.0,DW,16,90/100,387.83,0.893699,0.914790,2.891418,0.989443,0.959596,0.891599,0.891641,2.734122,0.999977,0.010535
4,20.0,DW,16,100/100,179.97,0.893699,0.914790,3.102420,0.990451,0.894378,0.891599,0.891641,2.734122,0.999977,0.009526
5,3.0,DW,16,100/100,238.07,0.893699,0.914790,2.924179,0.989613,0.985125,0.891599,0.891641,2.734122,0.999977,0.010364
6,4.0,DW,16,100/100,230.86,0.893699,0.914790,2.937113,0.989679,0.990421,0.891599,0.891641,2.734122,0.999977,0.010299
7,5.0,DW,16,100/100,240.60,0.893699,0.914893,2.941633,0.989651,0.990893,0.891599,0.891641,2.734122,0.999977,0.010326


In [4]:
best_values

[{'potential': 'QHO', 'cutoff': 2, 'best value': '10.0'},
 {'potential': 'QHO', 'cutoff': 4, 'best value': '5.0'},
 {'potential': 'QHO', 'cutoff': 8, 'best value': '2.0'},
 {'potential': 'QHO', 'cutoff': 16, 'best value': '2.0'},
 {'potential': 'AHO', 'cutoff': 2, 'best value': '5.0'},
 {'potential': 'AHO', 'cutoff': 4, 'best value': '5.0'},
 {'potential': 'AHO', 'cutoff': 8, 'best value': '5.0'},
 {'potential': 'AHO', 'cutoff': 16, 'best value': '3.0'},
 {'potential': 'DW', 'cutoff': 2, 'best value': '3.0'},
 {'potential': 'DW', 'cutoff': 4, 'best value': '2.0'},
 {'potential': 'DW', 'cutoff': 8, 'best value': '5.0'},
 {'potential': 'DW', 'cutoff': 16, 'best value': '10.0'}]

In [ ]:
ad = pd.DataFrame(alld)
ad['diff'] = ad[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
ad[(ad['potential']=='AHO') & (ad['cutoff']==16)].sort_values('diff')

In [ ]:
df

In [ ]:
df = pd.DataFrame(all_data)
df['diff'] = df[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
df.sort_values('diff')#.iloc[0]['final_tr_radius']

In [4]:
folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQD\Optimize\VaryBeta")
comp='beta'
potentials = ['QHO','AHO','DW']
cutoffs = [16]
converged_only = True
best_values = []
alld = []

for potential in potentials: 

    folders = [f for f in os.listdir(os.path.join(folder_path,potential)) if os.path.isdir(os.path.join(folder_path, potential, f))]

    for cutoff in cutoffs:
        all_data = []

        for folder in folders:   

            #print(f"Runing for {potential} potential, {cutoff} cutoff and {folder} value")
            
            median = []
            mean = []
            min = []
            e0 = []
            e1 = []
            e2 = []
            ratios = []

            data_path = os.path.join(folder_path, potential, str(folder), "{}_{}.json".format(potential, cutoff))
            
            with open(data_path, 'r') as file:
                data = json.load(file)

           
            num_VQD = data['num_VQD']

            converged = [all(x) for x in data['success']]
            converged_indices = [i for i, success in enumerate(converged) if success]
            converged_runs = sum(converged)

            if converged_only:
                results = [data['results'][i] for i in converged_indices]
            else:
                results = data['results']

            success_i = [i for i, success in enumerate(data['success']) if all(success)]
            successful_results = [data['results'][i] for i in success_i]
            energies =np.sort(successful_results)

            
            num_evals = data['num_evaluations']
            sum_evals = [sum(x) for x in num_evals]
            mean_evals = np.mean(sum_evals)

            num_iters = data['num_iters']
            sum_iters = [sum(x) for x in num_iters]
            mean_iters = np.mean(sum_iters)

            for e in np.sort(energies):
                e0.append(e[0])
                e1.append(e[1])
                e2.append(e[2])

            mean_ratio = abs((np.mean(e2) - np.mean(e1)) / (np.mean(e2) - np.mean(e0)))
            median_ratio = abs((np.median(e2) - np.median(e1)) / (np.median(e2) - np.median(e0)))

            e0_exact = data['exact_eigenvalues'][0]
            e1_exact = data['exact_eigenvalues'][1]
            e2_exact = data['exact_eigenvalues'][2]
            exact_ratio = abs((e2_exact - e1_exact) / (e2_exact - e0_exact))

            row = {
            comp: folder,
            'Converged Runs': f"{converged_runs}/{num_VQD}",
            r'$N_{\text{iters}}$': int(mean_iters),
            #r'$N_{\text{evals}}$': int(mean_evals),
            r'$R_{\text{exact}}$': exact_ratio,
            r'$R_{\text{med}}$': median_ratio
            }

            all_data.append(row)

        df = pd.DataFrame(all_data.copy())
        df['beta'] = df['beta'].astype(float)
        df[r"$\Delta R$"] = df[[r'$R_{\text{med}}$',r'$R_{\text{exact}}$']].apply(lambda x: np.abs(x[r'$R_{\text{med}}$'] - x[r'$R_{\text{exact}}$']), axis=1)
        df.sort_values("beta").to_latex(os.path.join(folder_path, potential, f"{potential}{cutoff}.tex"), index=False, float_format="%.4f")
        #df.sort_values(r"$\Delta R$").to_latex(os.path.join(folder_path, potential, f"{potential}{cutoff}.tex"), index=False)
        #df.to_csv(os.path.join(folder_path, potential, f"{potential}{cutoff}.csv"), float_format="%.10f")

       